# Better Than Vibes
    
While training and fine-tuning models models in a shoestring budget, the question of whether or not we had enough data was a constant problem. Is the problem with the model architecture? the implementation? or do we need more//better data? When I asked how we knew such and such data was enough the answer typically boiled down to "vibes." Let us see if we can do ***better than vibes.***

Marginalia provided in the code blocks.

## Introduction: Information Theory

Shannon information, $I$, is the measure of information a character in a message brings to a receiver. The information a character $a$ brings is written $$I(a) = -\log(p_a),$$ where $p_a$ is the probability that the character $a$ will be received. This probability is conditioned on the previous characters in the message *and* prior knowledge about the type of message. For example, a "u" following a "q" when the message is in English carries almost no information, since "u" nearly always follows "q" (conversely, any other letter carries large amounts of information, narrowing down the rest of the possible characters to only a handful). On the other hand, if the message were encrypted in such a way that all characters were equally likely, "u" after "q" would carry just as much information as any other character, them all having equal probability.

Turning back to machine learning, we can think of each sample in a test set as a message, beginning with the data and ending with the label(s). A model, then, is a machine that guesses the label(s), conditioned on the data *and* what it has seen during training. What we want a model to do is learn the patterns that allow one to infer the next part of a message, given the first part. A language model trained on English text would have learned that "u" nearly allows follows "q", for instance. We can then think of each *label* in a data set as holding a certain amount of information, depending on what the model has already learned. From here, we can start asking questions about how much data is enough and which data sets will be more useful for training.

In [6]:
# Import the module and get a dataset
import btv

# 44156 is the id for the "electricity" datasset, predicting whether the price of electricty rises or falls week to week.
# Feel free to chose another from the openML catalogue (https://www.openml.org/search?type=data&status=active)
# Make sure to update the name of the class column argument if necessary.
df, X, y = btv.data_tab.dataset2df(44156, class_cols=["class"], verbose=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

OpenML Dataset
Name..........: electricity
Version.......: 13
Format........: arff
Upload Date...: 2022-07-10 10:34:54
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/22103281/electricity.arff
OpenML URL....: https://www.openml.org/d/44156
# of features.: 9
# of instances: 38474


### Label information without any data
Without looking at the data or deploying any kind of model, the labels necessarily hold a lot of information. All we have to go on to guess the label of any given data point is the frequency of the label's occurrence. The information in each label, $y_i=c_i$, is written $$I(y_i) = -\log_2[p(y_i=c_i)]$$ where the probability of label $y_i$ being the correct class for the sample, $c_i$, can only be crudely estimated. Note that data of the sample, $x_i$, plays no role and the base of the logarithm makes the unit of information the bit.

In [7]:
naive_test_info = btv.core.chance_info(y_test,use_freq=False)
frequnecy_test_info = btv.core.chance_info(y_test,use_freq=True)
print("Information in the test set assuming p(label) == 1/# of classes:\n", naive_test_info)
print("Information in the test set assuming p(label) == 1/# of label ouccurances:\n", frequnecy_test_info)

Information in the test set assuming p(label) == 1/# of classes:
 7695.0
Information in the test set assuming p(label) == 1/# of label ouccurances:
 7694.988657169631


The frequency-aware information should be *lower* than the one based only on the number of classes, that reflecting the actual distribution of labels better. If they are close, that means the samples are very evenly distributed between the classes.
### Information remaining for a trained model
After training, a model will be able to use the data to infer the label. Therefore the information in the label for a trained model will be much less than using just frequency statistics. It can be written $$I(y_i|x_i) = -\log_2[p(y_i=c_i|x_i)],$$ where $p(y_i=c_i|x_i)$ is the probability estimation the classifier gives to the correct class. A *perfect* model would know every label with certainty from it's data, guess every label correctly with probability one, giving $I=0$ for all samples.

In [8]:
from sklearn.ensemble import GradientBoostingClassifier # a classic for tabulat data
# feel free to use another classifier. The methods called will be .fit(X,y), .predict_proba(X), and .score(X,y)
clf = GradientBoostingClassifier(
    n_estimators=100, learning_rate=1.0, max_depth=1, random_state=10
)
clf.fit(X_train, y_train)
predicted_label_probabilities = clf.predict_proba(X_test)
trained_test_info = btv.core.prediction_info(y_test, predicted_label_probabilities).sum() # function returns the information of each label
full_train_score = clf.score(X_test, y_test)

print(f"Information in the test set from frqunecy stats: {frequnecy_test_info:.3f}")
print(f"Information remaining in test set for trained classifier: {trained_test_info:.3f}")
print(f"Score when trained on full training set: : {full_train_score:0.5f}")

Information in the test set from frqunecy stats: 7694.989
Information remaining in test set for trained classifier: 1726.200
Score when trained on full training set: : 0.81442


Since the Shannon information of message can be thought of as the amount of information needed to specify the message, a better-trained model need less information about (or "leave less information in" if you use the thermodynamic analogy of information being hidden in a system) a test set.

In [9]:
# Select a minimum training set, one that only includes one of every label
min_idx, _ = btv.core.collect_min_set(y_train)
X_min, y_min = X_train[min_idx] , y_train[min_idx]

# Fitting to this minimum set will result in a functional, but poor classifier
clf.fit(X_min, y_min)
min_trained_predicted_test_probabilities = clf.predict_proba(X_test)
min_trained_test_info = btv.core.prediction_info(y_test, min_trained_predicted_test_probabilities).sum()
min_trained_score = clf.score(X_test,y_test)

print(f"Information in the test set from frqunecy stats: {frequnecy_test_info:.3f}")
print(f"Information remaining in test set for fully-trained classifier: {trained_test_info:.3f}")
print(f"Score when trained on full training set: : {full_train_score:0.5f}")
print(f"Information remaining in test set for minimum-trained classifier: {min_trained_test_info:.3f}")
print(f"Score when trained on minimum training set: : {min_trained_score:0.5f}")

Information in the test set from frqunecy stats: 7694.989
Information remaining in test set for fully-trained classifier: 1726.200
Score when trained on full training set: : 0.81442
Information remaining in test set for minimum-trained classifier: 2367.289
Score when trained on minimum training set: : 0.70292


The amount of information left in a test set is more than just anoter metric for classifier performance. The more ignorant a classifier is about a set, the more it can potentially learn from it.
#### Looking at the information remaining in the training set after training on subsets of it:

In [10]:
frequnecy_train_info = btv.core.chance_info(y_train)
clf.fit(X_train, y_train)
full_trained_train_info = btv.core.prediction_info(y_train, clf.predict_proba(X_train)).sum()
clf.fit(X_train[:len(y_train)//10], y_train[:len(y_train)//10])
tenth_trained_train_info = btv.core.prediction_info(y_train, clf.predict_proba(X_train)).sum()
clf.fit(X_min, y_min)
min_trained_train_info = btv.core.prediction_info(y_train, clf.predict_proba(X_train)).sum()

print(f"Information in the training set from frqunecy stats: {frequnecy_train_info:.3f}")
print(f"Information remaining in train set for minimum-trained classifier: {min_trained_train_info:.3f}")
print(f"Information remaining in train set for tenth-trained classifier: {tenth_trained_train_info:.3f}")
print(f"Information remaining in train set for fully-trained classifier: {full_trained_train_info:.3f}")

Information in the training set from frqunecy stats: 30778.997
Information remaining in train set for minimum-trained classifier: 9667.596
Information remaining in train set for tenth-trained classifier: 7132.725
Information remaining in train set for fully-trained classifier: 6953.905


And different training sets can have different amounts of information *about* the test set:

In [11]:
import numpy as np

idx_0s = np.where(y_train == 0)[0] 
idx_0splus1 = np.unique(np.append(min_idx, idx_0s)) # a training set of almost all zeros will not tell us much about the test set
clf.fit(X_train[idx_0splus1],y_train[idx_0splus1])
almost_all_zero_test_info = btv.core.prediction_info(y_test, clf.predict_proba(X_test)).sum()

print(f"Information in the test set from frqunecy stats: {frequnecy_test_info:.3f}")
print(f"Information remaining in test set for fully-trained classifier: {trained_test_info:.3f}")
print(f"Score when trained on full training set: : {full_train_score:0.5f}")
print(f"Information remaining in test set for minimum-trained classifier: {min_trained_test_info:.3f}")
print(f"Score when trained on minimum training set: : {min_trained_score:0.5f}")
print(f"Information remaining in test set for classifier trained on almost all 0 labels: {almost_all_zero_test_info:.3f}")
print(f"Score when trained on almost all 0 labels : {clf.score(X_test,y_test):0.5f}")

Information in the test set from frqunecy stats: 7694.989
Information remaining in test set for fully-trained classifier: 1726.200
Score when trained on full training set: : 0.81442
Information remaining in test set for minimum-trained classifier: 2367.289
Score when trained on minimum training set: : 0.70292
Information remaining in test set for classifier trained on almost all 0 labels: 3666.000
Score when trained on almost all 0 labels : 0.52359


NB: When using a random forest classifier on the electricity data set (OpenML id = 44156), the classifier trained on almost all zeros did *worse* than on trained on only two samples. It left more information in the test set ($\approx100$ bits) and scored $\approx1$% worse. This is not too surprizing since the training set with only one sample of each labels reflects the class balance of the test set.

It is tempting to say that we can answer the question posed in the first paragraph: if a fine-tuning set has less information for a model than the test set, it is insufficient. However, information scales with the size of the set and a small training set may contain all the paterns necissary to teach the model to classify a large test set well. Entropy (average information) is insensitive to class balance

# Training Data Selection
So Shnnon information is behaving as expected, giving a quanitative measure of how ignorant a classifier is about certain set of labels. If we are in the position of having to chose a training set however, information alone is not enough. Information a set contains for a given classifier scales with its size, while performance (and information remaining in a test set) once trained does not. What is more, ideally, we would select training data 

In [ ]:
# Chosing between fine-tuning sets
ds = data_tab.getdata(44156, verbose=False)
df, X, y = data_tab.dataset2df(ds, class_cols=["class"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = GradientBoostingClassifier(
    n_estimators=100, learning_rate=1.0, max_depth=1, random_state=10
)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_dev1, y_train, y_dev1 = train_test_split(X_train, y_train, test_size=0.2)
X_train, X_dev2, y_train, y_dev2 = train_test_split(X_train, y_train, test_size=0.2)
X_train, X_dev3, y_train, y_dev3 = train_test_split(X_train, y_train, test_size=0.2)
X_train, X_dev4, y_train, y_dev4 = train_test_split(X_train, y_train, test_size=0.2)
X_train, X_dev5, y_train, y_dev5 = train_test_split(X_train, y_train, test_size=0.2)

X_ft_list = [X_dev1, X_dev2, X_dev3, X_dev4, X_dev5]
y_ft_list = [y_dev1, y_dev2, y_dev3, y_dev4, y_dev5]